<a href="https://colab.research.google.com/github/gbend22/ml-homework/blob/main/experiments/N-Beats_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'ML_Final_Project/walmart-recruiting-store-sales-forecasting'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My\ Drive/$FOLDERNAME/

Mounted at /content/drive
/content/drive/My Drive/ML_Final_Project/walmart-recruiting-store-sales-forecasting


In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv('train.csv')
stores = pd.read_csv('stores.csv')
features = pd.read_csv('features.csv')
df = df.merge(stores, how='left', left_on='Store', right_on='Store')
df = df.merge(features, how='left', left_on=['Store','Date'], right_on=['Store','Date'])
df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,1,2010-02-12,46039.49,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,1,2010-02-19,41595.55,False,A,151315,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,1,2010-02-26,19403.54,False,A,151315,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,1,2010-03-05,21827.90,False,A,151315,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [3]:
!pip install pytorch-forecasting pytorch-lightning torchmetrics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.6/962.6 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.1/821.1 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [12]:
import torch
from pytorch_forecasting import TimeSeriesDataSet, NBeats, Baseline
from pytorch_forecasting.data import GroupNormalizer
from pytorch_lightning import Trainer
from sklearn.preprocessing import StandardScaler

df['Date'] = pd.to_datetime(df['Date'])

df = df.sort_values(['Store', 'Dept', 'Date'])

df.fillna(0, inplace=True)

df['time_idx'] = (df['Date'] - df['Date'].min()).dt.days

df['IsHoliday_y'] = df['IsHoliday_y'].astype(str)

scaler = StandardScaler()
for col in ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']:
    df[col] = scaler.fit_transform(df[[col]])

In [25]:
training_cutoff = df['time_idx'].max() - 6

training = TimeSeriesDataSet(
    df[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="Weekly_Sales",
    group_ids=["Store", "Dept"],

    max_encoder_length=30,
    max_prediction_length=6,

    time_varying_unknown_reals=["Weekly_Sales"],

    static_categoricals=[],
    time_varying_known_categoricals=[],

    time_varying_known_reals=[],

    target_normalizer=GroupNormalizer(groups=["Store", "Dept"]),
    add_relative_time_idx=False,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True
)

validation = training.to_dataloader(train=False, batch_size=128)

/usr/local/lib/python3.11/dist-packages/pytorch_forecasting/data/timeseries/_timeseries.py:1847: UserWarning: Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 133 series/groups which therefore are not present in the dataset index. This means no predictions can be made for those series. First 10 removed groups: [{'__group_id__Store': 1, '__group_id__Dept': 77}, {'__group_id__Store': 1, '__group_id__Dept': 78}, {'__group_id__Store': 2, '__group_id__Dept': 39}, {'__group_id__Store': 2, '__group_id__Dept': 77}, {'__group_id__Store': 3, '__group_id__Dept': 77}, {'__group_id__Store': 3, '__group_id__Dept': 78}, {'__group_id__Store': 3, '__group_id__Dept': 83}, {'__group_id__Store': 4, '__group_id__Dept': 39}, {'__group_id__Store': 5, '__group_id__Dept': 51}, {'__group_id__Store': 5, '__group_id__Dept': 78}]
  warnings.warn(


In [27]:
train_dataloader = training.to_dataloader(train=True, batch_size=128, num_workers=0)

model = NBeats.from_dataset(
    training,
    learning_rate=1e-3,
    log_interval=10,
    log_val_interval=1,
    weight_decay=1e-2,
    hidden_size=64,
    optimizer="adam"
)

trainer = Trainer(max_epochs=5, gradient_clip_val=0.1)

trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=validation)

AssertionError: The only variable as input should be the target which is part of time_varying_unknown_reals